# Personal Shopping Agent with Short-Term Memory

## Overview

This tutorial demonstrates how to build a personal shopping agent using Strands agents with AgentCore short-term memory. The agent remembers recent conversations in the session using `get_last_k_turns` and can continue conversations seamlessly when users return.

### Tutorial Details

| Information         | Details                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| Tutorial type       | Conversational                                                                   |
| Agent type          | Personal                                                                         |
| Agentic Framework   | Strands Agents                                                                   |
| LLM model           | Anthropic Claude Sonnet 3.7                                                      |
| Tutorial components | AgentCore Short-term Memory, AgentInitializedEvent hook                         |
| Tutorial vertical   | E-commerce                                                                       |
| Example complexity  | Beginner                                                                         |
| SDK used            | AgentCore Memory SDK and Strands Agents                                          |

You'll learn to:
- Use short-term memory without strategies
- Retrieve last K conversation turns
- Initialize agents with conversation history
- Handle session continuity across agent restarts

## Tutorial Architecture

```
┌─────────────────┐    ┌──────────────────┐    ┌─────────────────┐
│ User Returns    │───▶│ Personal Agent   │───▶│ Short-term      │
│ (Same Session)  │    │                  │    │ Memory          │
└─────────────────┘    │ ┌──────────────┐ │    │                 │
                       │ │ Initialized  │ │◄───┤ Last K Turns    │
                       │ │ Event Hook   │ │    │ (No Strategies) │
                       │ └──────────────┘ │    │                 │
                       └──────────────────┘    └─────────────────┘
```

### Key Features

- **Session Continuity**: Remembers recent conversation turns
- **No Strategies**: Uses raw conversation history without processing
- **Agent Initialization**: Loads context when agent starts
- **Personal Context**: Maintains shopping preferences and order history

## Prerequisites

- Python 3.10+
- AWS credentials with Amazon Bedrock AgentCore Memory permissions
- AgentCore Memory role ARN configured
- Amazon Bedrock AgentCore SDK
- Strands Agents framework

## Step-by-Step Tutorial

### Step 1: Install Dependencies and Setup

In [ ]:
import json
import logging
import os
import sys
import time
import uuid
import warnings
from datetime import datetime
from typing import Dict, List, Any, Optional

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s", datefmt="%H:%M:%S", stream=sys.stdout)
logger = logging.getLogger("memory-sample")
# Suppress urllib3 warnings and redirect stderr


region = "us-east-1"
current_directory = os.getcwd()

# sys.path.append(os.path.join(os.path.dirname("GenesisSDK"), './src'))
sys.path.append(os.path.join(current_directory,"BedrockAgentcoreSdkPython/src"))
path = os.path.join(current_directory,"BedrockAgentcoreSdkPython/src")
print(True if "bedrock_agentcore" in os.listdir(path) else False)

In [ ]:
import logging
import os
from typing import Dict, List
from datetime import datetime

# Setup logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger("personal-agent")

# Import required modules
from strands import Agent
from strands.hooks.events import AgentInitializedEvent, MessageAddedEvent
from strands.hooks.registry import HookProvider, HookRegistry
from bedrock_agentcore.memory import MemoryClient

# Configuration - Update with your values
REGION = "us-west-2"
ROLE_ARN = "arn:aws:iam::484907519812:role/service-role/AmazonSageMaker-ExecutionRole-20250219T174726"

#ROLE_ARN = "arn:aws:iam::YOUR_ACCOUNT:role/YOUR_GENESIS_ROLE"  # Replace with your role ARN
ACTOR_ID = "user_123"
SESSION_ID = "shopping_session_001"  # Reuse same session for continuity

### Step 2: Create Memory Resource

For short-term memory, we create a memory resource without any strategies. This stores raw conversation turns that can be retrieved with `get_last_k_turns`.

In [ ]:
# Initialize Memory Client
client = MemoryClient(region_name=REGION, environment="prod")
memory_name = "PersonalShoppingMemory"

# Create memory resource without strategies (short-term memory)
try:
    memory = client.create_memory_and_wait(
        name=memory_name,
        strategies=[],  # No strategies for short-term memory
        description="Short-term memory for personal shopping agent",
        event_expiry_days=7,  # Short retention period
        memory_execution_role_arn=ROLE_ARN,
    )
    memory_id = memory['memoryId']
    logger.info(f"✅ Created memory: {memory_id}")
except Exception as e:
    if "already exists" in str(e):
        memories = client.list_memories()
        memory_id = next((m['id'] for m in memories if m['id'].startswith(memory_name)), None)
        logger.info(f"✅ Using existing memory: {memory_id}")
    else:
        logger.error(f"Failed to initialize memory: {e}")
        raise e

### Step 3: Create Memory Hook for Agent Initialization

This hook retrieves the last K conversation turns when the agent is initialized, providing context from previous sessions.

In [ ]:
class ShortTermMemoryHook(HookProvider):
    def __init__(self, memory_client: MemoryClient, memory_id: str, actor_id: str, session_id: str):
        self.memory_client = memory_client
        self.memory_id = memory_id
        self.actor_id = actor_id
        self.session_id = session_id
    
    def on_agent_initialized(self, event: AgentInitializedEvent):
        """Load recent conversation history when agent starts"""
        try:
            # Get last 5 conversation turns
            recent_turns = self.memory_client.get_last_k_turns(
                memory_id=self.memory_id,
                actor_id=self.actor_id,
                session_id=self.session_id,
                k=5,
                branch_name="main"
            )
            
            if recent_turns:
                # Format conversation history for context
                context_messages = []
                for turn in recent_turns:
                    for message in turn:
                        role = message['role'].lower()
                        content = message['content']['text']
                        context_messages.append(f"{role.title()}: {content}")
                
                context = "\n".join(context_messages)
                logger.info(f"Context from memory: {context}")
                
                # Add context to agent's system prompt
                event.agent.system_prompt += f"\n\nRecent conversation history:\n{context}\n\nContinue the conversation naturally based on this context."
                
                logger.info(f"✅ Loaded {len(recent_turns)} recent conversation turns")
            else:
                logger.info("No previous conversation history found")
                
        except Exception as e:
            logger.error(f"Failed to load conversation history: {e}")
    
    def on_message_added(self, event):
        """Store conversation turns in memory"""
        try:
            print(f"message added ------- {event.message}")
            self.memory_client.save_conversation(
                memory_id=self.memory_id,
                actor_id=self.actor_id,
                session_id=self.session_id,
                messages=[(event.message["content"][0]["text"], event.message["role"])]
            )
            
        except Exception as e:
            logger.error(f"Failed to store message: {e}")
    
    def register_hooks(self, registry: HookRegistry) -> None:
        # Create and register memory hook
        registry.add_callback(MessageAddedEvent, self.on_message_added)
        registry.add_callback(AgentInitializedEvent, self.on_agent_initialized)
    

### Step 4: Create Personal Shopping Agent

In [ ]:
def create_personal_agent():
    """Create a personal shopping agent with short-term memory"""
    
    # Create agent
    agent = Agent(
        name="PersonalShoppingAssistant",
        system_prompt=f"""You are a helpful personal shopping assistant. You help users with:
        - Finding products and making purchase decisions
        - Tracking orders and managing purchases
        - Providing product recommendations
        - Answering questions about warranties and returns
        Today's date: {datetime.today().strftime('%Y-%m-%d')}
        Be friendly, helpful, and remember details from previous conversations.""",
        hooks=[ShortTermMemoryHook(client, memory_id, ACTOR_ID, SESSION_ID)]
    )
    return agent

# Create the agent
agent = create_personal_agent()
logger.info("✅ Personal shopping agent created with short-term memory")

### Step 5: Simulate First Shopping Session

Let's simulate an initial shopping conversation that will be stored in memory.

In [ ]:
# First conversation - user shares purchase information
print("=== First Shopping Session ===")

response1 = agent("I bought a new iPhone on 1st June 2025.")
print(f"User: I bought a new iPhone on 1st June 2025.")
print(f"Agent: {response1}\n")

In [ ]:
response2 = agent("Yes, the order number for my iPhone 15 Pro is 123456")
print(f"User: Yes, the order number for my iPhone 15 Pro is 123456")
print(f"Agent: {response2}\n")

In [ ]:
response3 = agent("I also bought new headphones from Sennheiser on 20th June 2025. Order number for that is 654321. They have a 1 year warranty.")
print(f"User: I also bought new headphones from Sennheiser on 20th June 2025. Order number for that is 654321. They have a 1 year warranty.")
print(f"Agent: {response3}\n")

In [ ]:
response4 = agent("I came to the e-store searching for laptops but didn't find anything for 2 hours")
print(f"User: I came to the e-store searching for laptops but didn't find anything for 2 hours")
print(f"Agent: {response4}\n")

### Step 6: Verify Memory Storage

Let's check what conversation turns are stored in memory.

In [ ]:
# Check stored conversation turns
print("=== Stored Conversation Turns ===")

recent_turns = client.get_last_k_turns(
    memory_id=memory_id,
    actor_id=ACTOR_ID,
    session_id=SESSION_ID,
    k=5,
)

for i, turn in enumerate(recent_turns, 1):
    print(f"Turn {i}:")
    for message in turn:
        role = message['role']
        content = message['content']['text']
        print(f"  {role}: {content}")
    print()

### Step 7: Simulate Agent Restart with Session Continuity

Now let's create a new agent instance (simulating the user returning) and see how it loads the conversation history.

In [ ]:
print("=== User Returns - New Agent Session ===")
print("Creating new agent instance (simulating user return)...\n")

# Create a new agent instance (user returns)
new_agent = create_personal_agent()

# The AgentInitializedEvent hook will automatically load recent conversation history
print("Agent initialized with conversation history loaded.\n")

# Continue the conversation
response = new_agent("Can you help me with my iPhone order?")
print(f"User: Can you help me with my iPhone order?")
print(f"Agent: {response}\n")

# Ask about previous purchases
response = new_agent("What was the warranty period for my headphones?")
print(f"User: What was the warranty period for my headphones?")
print(f"Agent: {response}\n")

### Step 8: Demonstrate Memory Limitations

Short-term memory only keeps the last K turns. Let's see what happens when we exceed that limit.

In [ ]:
print("=== Testing Memory Limits ===")

# Check what's still in memory (should only show last 5 turns)
print("=== Current Last 5 Turns ===")
recent_turns = client.get_last_k_turns(
    memory_id=memory_id,
    actor_id=ACTOR_ID,
    session_id=SESSION_ID,
    k=3
)

for i, turn in enumerate(recent_turns, 1):
    print(f"Turn {i}:")
    for message in turn:
        role = message['role']
        content = message['content']['text'][:50] + "..." if len(message['content']['text']) > 50 else message['content']['text']
        print(f"  {role}: {content}")
    print()

### Step 9: Cleanup (Optional)

In [ ]:
# Uncomment to delete the memory resource
# client.delete_memory(memory_id)
# logger.info(f"✅ Deleted memory: {memory_id}")

## Summary

This tutorial demonstrated how to build a personal agent with short-term memory using AgentCore Memory:

### Key Concepts Learned:

1. **Short-term Memory**: Created memory without strategies for raw conversation storage
2. **get_last_k_turns**: Retrieved recent conversation turns for context
3. **AgentInitializedEvent**: Used hooks to load conversation history when agent starts
4. **Session Continuity**: Maintained context across agent restarts using the same session_id
5. **Memory Limitations**: Understood that short-term memory only keeps the last K turns

### Use Cases for Short-term Memory:

- **Personal Assistants**: Remember recent conversations without long-term storage
- **Session-based Support**: Maintain context within a support session
- **Temporary Context**: Keep recent context without permanent memory strategies
- **Quick Interactions**: Fast context loading for brief conversations

### Next Steps:

- Explore long-term memory with strategies for persistent knowledge
- Combine short-term and long-term memory for comprehensive context
- Implement memory cleanup and management strategies
- Add more sophisticated conversation handling and context management